### Machine Learning: Naive Bayes classifier for categorical data from scratch in Python
$P(C_k|\textbf{x})\propto P(C_k) P(\textbf{x}|C_k)$
###### by Hamed Shah-Hosseini
Explanation at: https://www.pinterest.com/HamedShahHosseini/
<br>https://github.com/ostad-ai/Machine-Learning

In [1]:
import pandas as pd
from collections import defaultdict

In [28]:
# Reading dataset
weather=pd.read_csv('./datasets/weather.csv')
print(f'Number of samples in dataset:{len(weather)}')

Number of samples in dataset:14


In [29]:
# The first five rows of dataset
weather.head()

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rainy,Mild,High,False,Yes
4,Rainy,Cool,Normal,False,Yes


In [30]:
# naive Bayes for categorial data
#alpha is the smoothing parameter
class NBCategorial:
    def __init__(self,alpha=.5):
        self.alpha=alpha
    def fit(self,data_frame,class_column):
        df=data_frame.astype(str)
        self.groups=df.groupby(class_column)
        self.Ncats=dict() # no. of unique categories in each feature
        self.features=df.columns.to_list()
        self.features.remove(class_column)
        self.probs={key:defaultdict(dict) for key in self.features}
        for feature in self.features:
            self.Ncats[feature]=len(df[feature].unique())
        self.priors=(self.groups.size()/len(df)).to_dict() #prior
        for cName,cData in self.groups:            
            for feature in self.features:
                result=pd.value_counts(cData[feature]).to_dict()
                num=len(cData[feature])
                for value in df[feature].unique():
                    if value in result.keys():
                        self.probs[feature][value][cName]=\
                        (result[value]+self.alpha)/\
                       (num+self.alpha*self.Ncats[feature])
                    else: # value (category) is not in cData
                        self.probs[feature][value][cName]=self.alpha/\
                       (num+self.alpha*self.Ncats[feature])                
    def predict(self,query,with_priors=True):
        probs_classes=dict()        
        for cName,cData in self.groups:
            if with_priors:
                prob=self.priors[cName]
            else:
                prob=1.
            for q,feature in zip(query,self.features):
                if q in self.probs[feature].keys():
                    prob*=self.probs[feature][q][cName] 
                else: # haven't seen such a category
                    num=len(cData[feature])
                    prob*=self.alpha/(num+\
                    self.alpha*(1+self.Ncats[feature]))
            probs_classes[cName]=prob
        predict=max(probs_classes,key=probs_classes.get)
        return predict,probs_classes

In [31]:
# Testing with training data, not very usual:)
# That's because the dataset is so small
classCol='Play'
Ns=weather.shape[0] #number of samples
columns=weather.columns.to_list()
classCol_inx=columns.index(classCol)
nbayes=NBCategorial()
nbayes.fit(weather,classCol)
accuracy=0.
for n in range(Ns):
    query=weather.astype(str).iloc[n].to_list()
    #remove the value of class column
    class_q=query.pop(classCol_inx) 
    accuracy+=nbayes.predict(query)[0]==class_q
accuracy/=Ns    
print('Accuracy:%',round(100*accuracy,2))

Accuracy:% 92.86


In [23]:
# Example: testing with a given query
classCol='Play'
query=['Sunny','Mild','High','False']
nbayes=NBCategorial()
nbayes.fit(weather,classCol)
predict,probs=nbayes.predict(query)
print(f'Predicted class of {query}: {predict}')

Predicted class of ['Sunny', 'Mild', 'High', 'False']: No
